# Module 1.8: Data Preparation

> **Goal:** Transform first-contact data into a forecast-ready dataset: fill gaps, apply domain-appropriate imputation, and merge known-at-time calendar features.

By the end of this module, you'll have a dataset that is:
- **Continuous in time** — no missing weeks
- **Aligned to the business week** — Walmart's Sun-Sat fiscal week
- **Properly labeled with metadata** — department, category, state
- **Enriched with safe features** — known-at-time calendar attributes

| Step | What | Why |
|------|------|-----|
| 1 | Load data | Start from cleaned weekly data, diagnose gaps |
| 2 | Fill gaps | Complete weekly timeline for every series |
| 3 | Impute target | Apply domain-appropriate fill policy |
| 4 | Merge calendar | Add known-at-time features (weekly aggregated) |
| 5 | Document | Summarize all decisions and assumptions |

## Setup

In [1]:
# --- Imports ---
import sys
import warnings
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utilsforecast.preprocessing import fill_gaps
import forecast_foundations as ff
import tsforge as tsf

# --- Notebook Settings ---
env = ff.setup_notebook()

DATA_DIR = env.DATA_DIR
# OUTPUT_DIR = env.OUTPUT_DIR
cache = env.cache
output = env.output

✓ Setup complete | Root: real-world-forecasting-foundations | Notebook: 1.08_data_preparation | Data: /Users/lindsaytruong/forecast-academy/real-world-forecasting-foundations/data | Cache: on


---

## 1. Load Data

Load from Module 1.06 and understand what we're starting with.

### 1.1 Load from Module 1.6

In [2]:
weekly_sales, report_1_06 = output.load('1_06_first_contact', with_report=True)

✓ Loaded '1_06_first_contact'
   Shape: 6,848,638 × 8
   Report: ✓


In [3]:
report_1_06.summary     

{'Rows': '6,848,638',
 'Series': '30,490',
 'Dates': '2011-01-23 → 2016-06-19',
 'Frequency': 'Weekly',
 'History': '282 weeks (5.4 yrs)',
 'Target zeros': '24.0%'}

### 1.3 Save Hierarchy Data

Store hierarchy columns before `fill_gaps` — we'll rejoin them after.

In [4]:
# Identify hierarchy columns
hierarchy_cols = ['item_id', 'store_id', 'dept_id', 'cat_id', 'state_id']
hierarchy_cols = [c for c in hierarchy_cols if c in weekly_sales.columns]

In [5]:
# Create dimension table (one row per unique_id)
hierarchy_df = weekly_sales[['unique_id'] + hierarchy_cols].drop_duplicates(subset=['unique_id'])

In [6]:
hierarchy_df.head()

,unique_id,item_id,store_id,dept_id,cat_id,state_id
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,CA_1,HOBBIES_1,HOBBIES,CA
154,HOBBIES_1_001_CA_2,HOBBIES_1_001,CA_2,HOBBIES_1,HOBBIES,CA
307,HOBBIES_1_001_CA_3,HOBBIES_1_001,CA_3,HOBBIES_1,HOBBIES,CA
460,HOBBIES_1_001_CA_4,HOBBIES_1_001,CA_4,HOBBIES_1,HOBBIES,CA
614,HOBBIES_1_001_TX_1,HOBBIES_1_001,TX_1,HOBBIES_1,HOBBIES,TX


---

<div style="text-align: center;">

## 2. `Q3: Cadence` — Defines the Time Grid

<div style="background: linear-gradient(135deg, #2d42a7 0%, #3a2f7e 100%); color: white; padding: 12px 20px; border-radius: 8px; margin: 10px auto; max-width: 600px;">
<strong>Are the time intervals regular and complete?</strong><br>
<em>Gaps break lag features and corrupt rolling calculations.</em>
</div>

</div>

### 2.1 Check for Gaps

Use `datetime_diagnostics()` from `tsforge` to determine if we need to fill gaps.

In [7]:
# Run datetime diagnostics
date_table = tsf.datetime_diagnostics(
    df=weekly_sales,
    id_col="unique_id",
    date_col="ds",
    target_col="y",
)

date_table.head()

,start_date,end_date,n_obs,span_days,inferred_freq,obs_per_year,n_gaps,pct_missing,has_duplicates,peak_month,peak_quarter
unique_id,,,,,,,,,,,
FOODS_1_001_CA_1,2011-01-23,2016-06-19,283,1974.0,W-SUN,52.363602,0,0.0,False,5,2
FOODS_1_001_CA_2,2011-01-23,2016-06-19,283,1974.0,W-SUN,52.363602,0,0.0,False,6,2
FOODS_1_001_CA_3,2011-01-23,2016-06-19,283,1974.0,W-SUN,52.363602,0,0.0,False,2,1
FOODS_1_001_CA_4,2011-01-30,2016-06-19,282,1967.0,W-SUN,52.364260,0,0.0,False,3,3
FOODS_1_001_TX_1,2011-01-30,2016-06-19,282,1967.0,W-SUN,52.364260,0,0.0,False,5,1


In [8]:
# Check total gaps across all series
(date_table['n_gaps'] > 0).sum()

0

### 2.2 Fill Gaps

Run Nixtla's `fill_gaps` per `unique_id` to create a **complete weekly timeline** for every item-store pair. This inserts rows for missing dates with `y = NaN`.

In [9]:
# Determine week frequency from data
freq = pd.infer_freq(weekly_sales['week'].drop_duplicates().sort_values())
freq

KeyError: 'week'

In [ ]:
# before fill gaps
len(weekly_sales)

In [ ]:
weekly_sales_filled = fill_gaps(
    weekly_sales[['unique_id', 'ds', 'y']],
    freq=freq
)

len(weekly_sales_filled)

In [ ]:
weekly_sales_filled.sort_values(['unique_id', 'ds']).reset_index(drop=True, inplace=True)

In [ ]:
# Flag gap rows BEFORE imputation
weekly_sales_filled['is_gap'] = weekly_sales_filled['y'].isna().astype(int)
n_gaps = weekly_sales_filled['is_gap'].sum()

In [ ]:
#number of gaps to impute
n_gaps

### 2.3 Restore Metadata

When `fill_gaps` creates new weeks, metadata columns are empty. Re-attach using our stored dimension table.

In [ ]:
# Rejoin hierarchy
weekly_sales_filled = weekly_sales_filled.merge(
    hierarchy_df,
    on='unique_id',
    how='left'
)

In [ ]:
weekly_sales_filled

---

<div style="text-align: center;">

## 3. `Q1: Target` — Defines What We're Predicting

<div style="background: linear-gradient(135deg, #2596be 0%, #3a2f7e 100%); color: white; padding: 12px 20px; border-radius: 8px; margin: 10px auto; max-width: 600px;">
<strong>How do we treat missing target values?</strong><br>
<em>Imputation strategy depends on domain knowledge and business context.</em>
</div>

</div>

Now that we have a complete time grid with `y = NaN` for gap rows, we need to decide **how to fill those NaNs**. This is a business decision, not a technical one.

### 3.1 Imputation Strategies

| Strategy | How It Works | When to Use |
|----------|--------------|-------------|
| **Zero fill** | `y = 0` | Retail: store open, no sales recorded = no sales happened |
| **Forward fill** | Use last known value | Sensors: missing reading likely similar to previous |
| **Backward fill** | Use next known value | Late-arriving data: value exists, just delayed |
| **Interpolation** | Linear/spline between known points | Continuous processes: temperature, stock prices |
| **Seasonal fill** | Use same period last year/cycle | Strong seasonality: tourism, agriculture |
| **Mean/Median fill** | Use series average | When no pattern exists, need a neutral value |
| **Model-based** | Predict missing values | When you have reliable exogenous features |
| **Leave as NaN** | Don't impute | When downstream model handles missingness |

### 3.2 M5 Fill Policy

For M5 retail data, we apply:

| Scenario | Data Pattern | Policy |
|----------|--------------|--------|
| Store open, item available, no one bought it | Gap in middle of active series | `y = 0` |
| Item newly stocked, slow start | Gap at series start | `y = 0` |
| Item still active, just no sales that week | Gap at series end | `y = 0` |
| Store closed (holiday, renovation) | Could exclude from training or keep as 0 | Flag separately |
| Item discontinued | Zeros followed by series end | `y = 0` until discontinuation |

In [ ]:
# Apply fill policy: missing weeks → 0 (retail assumption: no sale = 0 units)
weekly_sales_filled['y'] = weekly_sales_filled['y'].fillna(0)

In [ ]:
# Verify no NaNs remain
n_gaps = weekly_sales_filled['y'].isna().sum()

n_gaps

In [ ]:
weekly_sales_filled.drop(columns=['is_gap'], inplace=True)

---

<div style="text-align: center;">

## 4. `Q4: Data` — Defines What the Model Learns

<div style="background: linear-gradient(135deg, #1d1f56 0%, #2d42a7 100%); color: white; padding: 12px 20px; border-radius: 8px; margin: 10px auto; max-width: 600px;">
<strong>What features can we safely add without leakage?</strong><br>
<em>Calendar features are known-at-time — safe for any forecast date.</em>
</div>

</div>

The `calendar.csv` file is **daily**, but our sales are **weekly**. We need to:
1. Determine week alignment (start vs end)
2. Create a matching week column in calendar
3. Aggregate daily features to weekly

### 4.1 Determine Week Alignment

Our `ds` column contains Sundays (e.g., 2011-01-23). 

**Walmart's fiscal week runs Sunday to Saturday**, so Sunday is the **week start**:
- `ds = 2011-01-23` represents sales from Sun 2011-01-23 through Sat 2011-01-29

### 4.2 Create Week Column in Calendar

In [ ]:
# Load calendar
calendar = load_m5_calendar(DATA_DIR)
calendar['date'] = pd.to_datetime(calendar['date'])

calendar.head()

In [ ]:
#Walmart's fiscal week runs Sunday to Saturday**, so Sunday is the **week start
calendar['week_start'] = calendar['date'] - pd.to_timedelta((calendar['date'].dt.dayofweek + 1) % 7, unit='D')
calendar[['date', 'week_start', 'weekday']].head(14)

### 4.3 Aggregate Calendar to Weekly

| Feature | Aggregation Rule |
|---------|------------------|
| Holiday/Event | If any day in week has it → week has it |
| SNAP flags | Same — max within week |
| wm_yr_wk | From **first day of week** (Sunday) |
| Year, month | From **first day of week** (Sunday) |

Use the `aggregate_calendar_to_weekly()` utility function to aggregate daily calendar data to weekly.

This function handles:
1. Groups daily rows by week (Sunday = week start for Walmart)
2. Takes first value for calendar IDs (wm_yr_wk, month, year)  
3. Collects all unique events and splits them into separate columns
4. Uses max() for SNAP flags (1 if ANY day in week had benefits)

In [ ]:
# Aggregate daily calendar to weekly using our utility function
# The result has one row per week with event_name_1, event_name_2, etc.
# based on however many events occurred in the week

weekly_calendar = aggregate_calendar_to_weekly(calendar)

weekly_calendar.head()

### 4.4 Merge into Sales

In [ ]:
weekly_sales_filled.shape

In [ ]:
merged_df = weekly_sales_filled.merge(
    weekly_calendar,
    on='ds',
    how='left'
)

In [ ]:
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
# Final column check
merged_df.columns

### 4.5 What We're NOT Adding (Yet)

| Feature | Why Excluded | When to Add |
|---------|--------------|-------------|
| Price features | Requires lagging to avoid leakage | Feature Engineering module |
| Lag features | Created during model training | Modeling module |
| Outlier flags | Need baseline forecast first | Post-baseline module |

---

<div style="text-align: center;">

## 5. `Q5: Ownership` — Defines Transparency

<div style="background: linear-gradient(135deg, #1d1f56 0%, #3a2f7e 100%); color: white; padding: 12px 20px; border-radius: 8px; margin: 10px auto; max-width: 600px;">
<strong>What assumptions are baked into this data?</strong><br>
<em>Document decisions so downstream users can trace and adjust.</em>
</div>

</div>

### 5.1 All Decisions Made - What We Did

| Step | Decision | Assumption | Reversible? |
|------|----------|------------|-------------|
| Gap detection | Used `datetime_diagnostics` | Weekly frequency is correct | ✓ Re-run with different freq |
| Gap filling | Nixtla `fill_gaps` | Series should span full date range | ✓ `is_gap` flag preserved |
| Imputation | Zero fill for all gaps | Missing = no sales (retail) | ✓ Can re-impute using `is_gap` |
| Calendar aggregation | Events: any in week | One event day = event week | ✓ Raw calendar available |
| Calendar aggregation | SNAP: max in week | One SNAP day = SNAP week | ✓ Raw calendar available |
| Calendar aggregation | Fiscal: first day of week | Week inherits Sunday's attributes | ✓ Can change to last/mode |
| Hierarchy | Static per unique_id | Items don't change department | — |

### 5.2 What Changed - The Result

In [ ]:
report_1_08 = first_contact_check(
    merged_df,
    dataset_name='Merged Weekly Sales and Calendar',
    prior_report=report_1_06
)

In [ ]:
report_1_08.changes()

### 5.3 Save

In [ ]:
# Cache for downstream modules
artifacts.save(
    df=merged_df,
    report=report_1_08
)

## Next Steps

| Module | Focus |
|--------|-------|
| **1.11** | Plotting & visual diagnostics |
| **2.1** | Baseline models — naive, seasonal naive |
| **2.2** | Statistical models — ETS, ARIMA, Theta |
| **2.3** | Feature engineering — price features with proper lagging |